In [9]:
from __future__ import absolute_import, print_function
import tweepy
import json
import os
import pandas as pd
from tweepy.streaming import StreamListener
import pprint as pp

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [10]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
consumer_token = os.getenv("access_token")
access_token_secret = os.getenv("access_token_secret")

In [11]:
def authorize_twitter_api():
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    return auth

In [12]:
api = tweepy.API(auth)

NameError: name 'auth' is not defined

In [6]:
for tweet in tweepy.Cursor(api.search, q='#contentmarketing', count=20000, 
    lang='en').items():
        tweets.append(tweet)

NameError: name 'api' is not defined

In [ ]:
#mike whats our count?

In [ ]:
def twitter_api_call(search_term,tweet_mode = 'extended',count = 5, lang='en',  **kwargs):
    """Makes twitter api call using tweepy"""
    tweets  = api.search(
        q = search_term,
        tweet_mode = tweet_mode,
        count =  count,
        lang = lang, 
        
    
    
    
    
    )
    return tweets

## Tokenizer Function 

In [ ]:
# tokenizer function
lemmatizer = WordNetLemmatizer()
sw = set(stopwords.words('english'))  
#adding custom stopwords to our stopwords set
custom_sw =['char','chars','say']
sw.update(custom_sw)
regex = re.compile(r"[^a-zA-Z ]")

In [ ]:

def clean_tokens(text):
    """Cleans and tokenizes text using nltk stopwords, regex to remove symols, lemmatizer."""
    #Apply regex filter 
    regex_filter = regex.sub('', text)
    #Get Tokens
    tokens = word_tokenize(regex_filter)
    #Set to lowercase if not in Stop Words list
    lowered_sw = [word.lower() for word in tokens if word.lower() not in sw]
    #get lemmatized clean tokens
    clean_tokens = [lemmatizer.lemmatize(word) for word in lowered_sw]
    return clean_tokens

In [ ]:
def news_api_df(articles_response,search_term,tokenize=False,sentiment=False,**kwargs):
    
  

        articles_list = []

        for article in articles_response['articles']:
            try:
                articles_list.append({
                    "date":article['publishedAt'][:10],
                    "source":article['source']['name'],
                    "url":article['url'],
                    "author":article['author'],
                    "text":article['content'],
                    "search_term":search_term
                })

            except AttributeError:
                pass

        articles_df = pd.DataFrame(articles_list)

        columns = ['date','search_term','source','url','author','text']

        articles_df = articles_df[columns]

        articles_df['text'] = articles_df['text'].astype(str)

        if tokenize:

            articles_df['clean_tokens'] = articles_df['text'].apply(clean_tokens)

            if sentiment:

                sentiment_score_list = []

                for article in articles_df['clean_tokens']:
                    string = " ".join(article)
                    sentiment_score = analyzer.polarity_scores(string)
                    sentiment_score_list.append(sentiment_score)

                sentiment_df = pd.DataFrame(sentiment_score_list)

                sentiment_df.columns = ['compound','negative','neutral','positive']

                articles_df = pd.concat([articles_df, sentiment_df], axis=1)

                return articles_df

            else:

                return articles_df

        if sentiment:

            sentiment_score_list = []

            for article in articles_df['text']:
                sentiment_score = analyzer.polarity_scores(article)
                sentiment_score_list.append(sentiment_score)

            sentiment_df = pd.DataFrame(sentiment_score_list)

            sentiment_df.columns = ['compound','negative','neutral','positive']

            articles_df = pd.concat([articles_df, sentiment_df], axis=1)

            return articles_df

        else:

            return articles_df


In [ ]:
def sentiment_df(search_term,**kwargs):

    """
    Wrapper for news_api_call, clean_tokens, and news_api_df
    most common syntax is df = sentiment_df("search_term",tokenize=True,sentiment=True)
    Optional Parameters for api call: language(default="en"), page_size(default=100)
    sort_by(default="relevancy"),sources(default=None),domains(default=None)
    
    """

    
    return news_api_df(news_api_call(search_term,**kwargs),search_term,**kwargs)
